In [1]:
from pyecharts.charts import Line, Bar, Pie, Grid
from pyecharts import options as opts
import pandas as pd
import numpy as np
#载入数据
today_province= pd.read_csv('./原始数据/today_province_2020_04_01.csv')
today_world= pd.read_csv('./原始数据/today_world_2020_04_01.csv')
alltime_province= pd.read_csv('./原始数据/alltime_province_2020_04_01.csv')
alltime_world= pd.read_csv('./原始数据/alltime_world_2020_04_01.csv')
alltime_China= pd.read_csv('./原始数据/alltime_China_2020_03_30.csv')

In [2]:
today_world.head()

,id,lastUpdateTime,name,total_confirm,total_suspect,total_heal,total_dead,total_severe,total_input,today_confirm,today_suspect,today_heal,today_dead,today_severe,today_storeConfirm,today_input
0,9577772,2020-04-01 07:38:32,突尼斯,394,0,3,10,0,NaN,32.0,0.0,0.0,2.0,0.0,NaN,NaN
1,9507896,2020-04-01 12:21:07,塞尔维亚,900,0,0,23,0,NaN,115.0,0.0,0.0,7.0,0.0,NaN,NaN
2,0,2020-04-01 14:31:31,中国,82640,183,76417,3321,466,806.0,95.0,0.0,192.0,7.0,-62.0,NaN,35.0
3,1,2020-04-01 14:02:32,日本,2945,0,1043,77,0,NaN,22.0,NaN,0.0,0.0,NaN,NaN,NaN
4,2,2020-04-01 14:02:04,泰国,1771,0,127,12,0,NaN,120.0,0.0,0.0,2.0,0.0,NaN,NaN


In [3]:
#世界各国实时数据处理
name_dict = {'date':'日期','name':'名称','id':'编号','lastUpdateTime':'更新时间',
             'today_confirm':'当日新增确诊','today_suspect':'当日新增疑似',
             'today_heal':'当日新增治愈','today_dead':'当日新增死亡',
             'today_severe':'当日新增重症','today_storeConfirm':'当日现存确诊',
             'total_confirm':'累计确诊','total_suspect':'累计疑似',
             'total_heal':'累计治愈','total_dead':'累计死亡','total_severe':'累计重症','total_input':'累计境外输入','today_input':'当日境外输入'}

# 更改列名
today_world.rename(columns= name_dict ,inplace=True)    # inplace参数判断是否在原数据上进行修改

today_world.head(3)

,编号,更新时间,名称,累计确诊,累计疑似,累计治愈,累计死亡,累计重症,累计境外输入,当日新增确诊,当日新增疑似,当日新增治愈,当日新增死亡,当日新增重症,当日现存确诊,当日境外输入
0,9577772,2020-04-01 07:38:32,突尼斯,394,0,3,10,0,NaN,32.0,0.0,0.0,2.0,0.0,NaN,NaN
1,9507896,2020-04-01 12:21:07,塞尔维亚,900,0,0,23,0,NaN,115.0,0.0,0.0,7.0,0.0,NaN,NaN
2,0,2020-04-01 14:31:31,中国,82640,183,76417,3321,466,806.0,95.0,0.0,192.0,7.0,-62.0,NaN,35.0


In [4]:
# 计算缺失值比例
today_world_nan = today_world.isnull().sum()/len(today_world)

# 转变为百分数
today_world_nan.apply(lambda x: format(x, '.1%')) 

编号          0.0%
更新时间        0.0%
名称          0.0%
累计确诊        0.0%
累计疑似        0.0%
累计治愈        0.0%
累计死亡        0.0%
累计重症        0.0%
累计境外输入     99.5%
当日新增确诊     40.0%
当日新增疑似     16.0%
当日新增治愈     40.0%
当日新增死亡     40.0%
当日新增重症     16.0%
当日现存确诊    100.0%
当日境外输入     99.5%
dtype: object

In [5]:
# 缺失值处理
#当日现存确诊一列虽然全部为空，但该缺失值可以通过已有数据直接计算，公式为：当日现存确诊=累计确诊−累计治愈−累计死亡
today_world['当日现存确诊'] = today_world['累计确诊']-today_world['累计治愈']-today_world['累计死亡']
today_world['病死率'] = (today_world['累计死亡']/today_world['累计确诊']).apply(lambda x: format(x, '.2f')) 

# 将病死率数据类型转换为float
today_world['病死率'] = today_world['病死率'].astype('float')

# 根据病死率降序排序
today_world.sort_values('病死率',ascending=False,inplace=True)

today_world.to_csv("today_world_to4.1.csv", index=False, sep=',')
# 显示病死率前十国家
today_world.head(10)
#病死率应结合累计确诊人数一起查看,不然无太大意义

,编号,更新时间,名称,累计确诊,累计疑似,累计治愈,累计死亡,累计重症,累计境外输入,当日新增确诊,当日新增疑似,当日新增治愈,当日新增死亡,当日新增重症,当日现存确诊,当日境外输入,病死率
139,95291,2020-04-01 12:34:51,苏丹,7,0,0,2,0,NaN,1.0,0.0,0.0,0.0,0.0,5,NaN,0.29
183,956789,2020-03-31 14:26:05,安哥拉,7,0,1,2,0,NaN,NaN,0.0,NaN,NaN,0.0,4,NaN,0.29
117,8197,2020-04-01 08:38:13,博茨瓦纳,4,0,0,1,0,NaN,1.0,0.0,0.0,1.0,0.0,3,NaN,0.25
124,95783,2020-03-30 09:33:45,冈比亚,4,0,0,1,0,NaN,NaN,0.0,NaN,NaN,0.0,3,NaN,0.25
123,95787,2020-03-29 12:40:40,尼加拉瓜,4,0,0,1,0,NaN,NaN,0.0,NaN,NaN,0.0,3,NaN,0.25
184,87656,2020-03-27 11:32:45,开曼群岛,5,0,0,1,0,NaN,NaN,0.0,NaN,NaN,0.0,4,NaN,0.20
78,8134,2020-04-01 08:07:49,毛里塔尼亚,6,0,2,1,0,NaN,1.0,0.0,0.0,1.0,0.0,3,NaN,0.17
120,95306,2020-03-31 00:00:31,佛得角,6,0,0,1,0,NaN,NaN,0.0,NaN,NaN,0.0,5,NaN,0.17
133,856671,2020-03-31 00:00:31,荷属安的列斯,7,0,0,1,0,NaN,NaN,0.0,NaN,NaN,0.0,6,NaN,0.14
159,15,2020-04-01 00:52:03,意大利,105792,0,15729,12428,0,NaN,4053.0,NaN,1109.0,837.0,NaN,77635,NaN,0.12


In [6]:
#为了方便查询特定国家的数据，我们使用set_index()函数将国家设置为索引
# 将国家名称设为索引
today_world.set_index('名称',inplace=True)

today_world.head(3)

,编号,更新时间,累计确诊,累计疑似,累计治愈,累计死亡,累计重症,累计境外输入,当日新增确诊,当日新增疑似,当日新增治愈,当日新增死亡,当日新增重症,当日现存确诊,当日境外输入,病死率
名称,,,,,,,,,,,,,,,,
苏丹,95291,2020-04-01 12:34:51,7,0,0,2,0,NaN,1.0,0.0,0.0,0.0,0.0,5,NaN,0.29
安哥拉,956789,2020-03-31 14:26:05,7,0,1,2,0,NaN,NaN,0.0,NaN,NaN,0.0,4,NaN,0.29
博茨瓦纳,8197,2020-04-01 08:38:13,4,0,0,1,0,NaN,1.0,0.0,0.0,1.0,0.0,3,NaN,0.25


In [7]:
# 查看某一国家的数据
today_world.loc['美国']    #可以通过传入列表获取多个国家的数据

编号                          7
更新时间      2020-04-01 11:57:51
累计确诊                   189510
累计疑似                        0
累计治愈                     7109
累计死亡                     3900
累计重症                        0
累计境外输入                    NaN
当日新增确诊                  25256
当日新增疑似                      0
当日新增治愈                   1564
当日新增死亡                    947
当日新增重症                      0
当日现存确诊                 178501
当日境外输入                    NaN
病死率                      0.02
Name: 美国, dtype: object

In [8]:
# 查看当前累计确诊人数前十国家
world_top10 = today_world.sort_values(['累计确诊'],ascending=False)[:10]

world_top10 = world_top10[['累计确诊','累计死亡','病死率']]

world_top10

,累计确诊,累计死亡,病死率
名称,,,
美国,189510,3900,0.02
意大利,105792,12428,0.12
西班牙,95923,8464,0.09
中国,82640,3321,0.04
德国,71808,775,0.01
法国,52836,3532,0.07
伊朗,44606,2898,0.06
英国,25481,1793,0.07
瑞士,16605,433,0.03


In [9]:
print(list(world_top10.index))
print(list(world_top10['累计确诊']))
print(list(world_top10['累计死亡']))
print(list(world_top10['病死率']))

['美国', '意大利', '西班牙', '中国', '德国', '法国', '伊朗', '英国', '瑞士', '土耳其']
[189510, 105792, 95923, 82640, 71808, 52836, 44606, 25481, 16605, 13531]
[3900, 12428, 8464, 3321, 775, 3532, 2898, 1793, 433, 214]
[0.02, 0.12, 0.09, 0.04, 0.01, 0.07, 0.06, 0.07, 0.03, 0.02]


In [10]:
v1 = [189510, 105792, 95923, 82640, 71808, 52836, 44606, 25481, 16605, 13531]
v2 = [3900, 12428, 8464, 3321, 775, 3532, 2898, 1793, 433, 214]
v3 = [0.02, 0.12, 0.09, 0.04, 0.01, 0.07, 0.06, 0.07, 0.03, 0.02]
bar = (
    Bar()
    .add_xaxis(xaxis_data=['美国', '意大利', '西班牙', '中国', '德国', '法国', '伊朗', '英国', '瑞士', '土耳其'])
    .add_yaxis("累计确诊",v1)
    .add_yaxis("累计死亡",v2)
    .set_series_opts(label_opts=opts.LabelOpts(is_show=True))
    .extend_axis(
        yaxis=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(formatter="{value} %")
        )
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="当前累计确诊人数前十国家"),
        yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="{value} ml")),
    )
)

line = Line().add_xaxis(xaxis_data=['美国', '意大利', '西班牙', '中国', '德国', '法国', '伊朗', '英国', '瑞士', '土耳其']).add_yaxis("病死率",v3, yaxis_index=1)
bar.overlap(line)
bar.render("当前累计确诊人数前十国家.html")
bar.render_notebook()

In [11]:
#2.全国各省实时数据探索性分析
# 创建中文列名字典
name_dict = {'date':'日期','name':'名称','id':'编号','lastUpdateTime':'更新时间',
             'today_confirm':'当日新增确诊','today_suspect':'当日新增疑似',
             'today_heal':'当日新增治愈','today_dead':'当日新增死亡',
             'today_severe':'当日新增重症','today_storeConfirm':'当日现存确诊',
             'total_confirm':'累计确诊','total_suspect':'累计疑似',
             'total_heal':'累计治愈','total_dead':'累计死亡','total_severe':'累计重症','total_input':'累计境外输入','today_input':'当日境外输入'}

# 更改列名
today_province.rename(columns=name_dict,inplace=True)    # inplace参数是否在原对象基础上进行修改

today_province.head()

,编号,更新时间,名称,累计确诊,累计疑似,累计治愈,累计死亡,累计重症,累计境外输入,当日新增确诊,当日新增疑似,当日新增治愈,当日新增死亡,当日新增重症,当日现存确诊,当日境外输入
0,420000,2020-04-01 13:28:00,湖北,67802,0,63326,3193,0,0,1.0,NaN,173.0,6.0,NaN,NaN,0.0
1,440000,2020-04-01 13:11:48,广东,1501,0,1357,8,0,0,11.0,NaN,4.0,0.0,NaN,NaN,0.0
2,410000,2020-04-01 13:50:22,河南,1276,0,1251,22,0,0,0.0,NaN,0.0,0.0,NaN,NaN,0.0
3,330000,2020-04-01 09:03:23,浙江,1257,0,1226,1,0,0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
4,430000,2020-04-01 11:01:44,湖南,1018,0,1014,4,0,0,0.0,NaN,0.0,0.0,NaN,NaN,0.0


In [12]:
# 查看数值型特征的统计量
today_province.describe()

,编号,累计确诊,累计疑似,累计治愈,累计死亡,累计重症,累计境外输入,当日新增确诊,当日新增疑似,当日新增治愈,当日新增死亡,当日新增重症,当日现存确诊,当日境外输入
count,34.000000,34.000000,34.0,34.000000,34.000000,34.0,34.0,31.000000,4.0,31.000000,31.000000,4.0,0.0,31.0
mean,422941.176471,2430.588235,0.0,2247.558824,97.676471,0.0,0.0,3.000000,0.0,6.193548,0.225806,0.0,NaN,0.0
std,195021.823359,11557.856924,0.0,10799.540608,546.947270,0.0,0.0,7.042727,0.0,30.988621,1.086575,0.0,NaN,0.0
min,110000.000000,1.000000,0.0,1.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,NaN,0.0
25%,312500.000000,138.500000,0.0,124.250000,1.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,NaN,0.0
50%,425000.000000,326.000000,0.0,246.000000,3.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,NaN,0.0
75%,537500.000000,697.000000,0.0,615.750000,6.000000,0.0,0.0,2.000000,0.0,0.000000,0.000000,0.0,NaN,0.0
max,820000.000000,67802.000000,0.0,63326.000000,3193.000000,0.0,0.0,32.000000,0.0,173.000000,6.000000,0.0,NaN,0.0


In [13]:
#累计疑似、累计重症、当日新增疑似和当日新增重症的数据全部为0，因此我们便不予考虑
# 计算各省当日现存确诊人数
today_province['当日现存确诊'] = today_province['累计确诊']-today_province['累计治愈']-today_province['累计死亡']
today_province.to_csv("today_province_to4.1.csv", index=False, sep=',')
# 将各省名称设置为索引
today_province.set_index('名称',inplace=True)

today_province.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34 entries, 湖北 to 西藏
Data columns (total 15 columns):
编号        34 non-null int64
更新时间      34 non-null object
累计确诊      34 non-null int64
累计疑似      34 non-null int64
累计治愈      34 non-null int64
累计死亡      34 non-null int64
累计重症      34 non-null int64
累计境外输入    34 non-null int64
当日新增确诊    31 non-null float64
当日新增疑似    4 non-null float64
当日新增治愈    31 non-null float64
当日新增死亡    31 non-null float64
当日新增重症    4 non-null float64
当日现存确诊    34 non-null int64
当日境外输入    31 non-null float64
dtypes: float64(6), int64(8), object(1)
memory usage: 4.2+ KB


In [14]:
#2.2全国新增确诊top10地区
new_top6 = today_province['当日新增确诊'].sort_values(ascending=False)[:10]
locate1 = list(new_top6.index)
data1 = list(new_top6.values)
map_1 = (
        Bar()
        .add_xaxis(locate1)
        .add_yaxis('',data1)
        .reversal_axis()
        .set_series_opts(label_opts=opts.LabelOpts(position="right"))
        )
    
map_2 = (
    Pie() #创建一个饼图init_opts=opts.InitOpts(width="700px", height="500px")可自定义
    .add(
        "4月1日全国新增确诊top10地区", #图名
        [[locate1[i],data1[i]] for i in range(len(locate1))], #添加数据
        radius=["40%", "75%"], # 调整半径
    )
    .set_global_opts(
        legend_opts=opts.LegendOpts( 
            orient="vertical", pos_top="10%", pos_left="88%" #图例设置
        ),
        title_opts=opts.TitleOpts(title="全国新增确诊top10地区",subtitle='2020-4-1'
        ),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        toolbox_opts=opts.ToolboxOpts(is_show=True),
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}例/{d}%")) #设置标签
)
grid=Grid()#init_opts=opts.InitOpts()
grid.add(map_1, opts.GridOpts(pos_top="50%", pos_right="75%"), is_control_axis_index=True)
grid.add(map_2, opts.GridOpts(pos_left="60%"), is_control_axis_index=True)
grid.render("全国新增确诊top10地区.html")
grid.render_notebook()


#new_top6 = today_province['当日新增确诊'].sort_values(ascending=False)[:10]
# 绘制条形图和饼图
#bar =Bar("x 轴和 y 轴交换")
#new_top6.sort_values(ascending=True).plot.barh(fontsize=10,ax=ax[0])
#new_top6.plot.pie(autopct='%.1f%%',fontsize=10,ax=ax[1])
#plt.ylabel('')
#plt.title('4月1日全国新增确诊top10地区',size=15)
#plt.show()                                                                                                                                  

In [15]:
store_top10=today_province['当日现存确诊'].sort_values(ascending=False)[:10]
locate2 = list(store_top10.index)
data2 = list(store_top10.values)
print(locate2)
print(data2)

['湖北', '香港', '台湾', '上海', '北京', '广东', '福建', '天津', '内蒙古', '澳门']
[1283, 582, 285, 167, 154, 136, 47, 38, 36, 31]


In [16]:
#2.3全国现存确诊人数top10的地区
# 查看全国现存确诊人数top10的省市
from pyecharts.charts import  Bar,Line
from pyecharts import options as opts
#store_top10 = today_province['当日现存确诊'].sort_values(ascending=False)[:10]
#locate2 = list(store_top10.index)
#data2 = list(store_top10.values)
map_3=(
        Line()
        .add_xaxis(xaxis_data=['湖北', '香港', '台湾', '上海', '北京', '广东', '福建', '天津', '内蒙古', '澳门'])
        .add_yaxis('柱形',[1283, 582, 285, 167, 154, 136, 47, 38, 36, 31])
        .set_global_opts(
            title_opts=opts.TitleOpts(title="全国现存确诊top10地区",subtitle='2020-4-1'),
            yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="{value} 例")),
            toolbox_opts=opts.ToolboxOpts(is_show=True),
        )
)
map_3.render("全国现存确诊人数top10的地区.html")
map_3.render_notebook()
# matplotlib绘制条形图
#store_top10.sort_values(ascending=True).plot.barh(fontsize=10)
#plt.title('全国现存确诊top10地区',size=15)
#plt.show()

In [17]:
#3. 历史数据探索性分析
# 创建中文列名字典
name_dict = {'date':'日期','name':'名称','id':'编号','lastUpdateTime':'更新时间',
             'today_confirm':'当日新增确诊','today_suspect':'当日新增疑似',
             'today_heal':'当日新增治愈','today_dead':'当日新增死亡',
             'today_severe':'当日新增重症','today_storeConfirm':'当日现存确诊',
             'total_confirm':'累计确诊','total_suspect':'累计疑似',
             'total_heal':'累计治愈','total_dead':'累计死亡','total_severe':'累计重症'}

# 更改列名
alltime_China.rename(columns=name_dict,inplace=True)

alltime_China.head()

,日期,累计确诊,累计死亡,累计治愈,累计重症,累计疑似,当日新增确诊,当日新增死亡,当日新增治愈,当日新增重症,当日现存确诊,当日新增疑似
0,2020/1/20,291,6,25,0,54,291,6,25,0,NaN,27
1,2020/1/21,440,9,25,102,37,149,3,0,0,NaN,26
2,2020/1/22,571,17,28,95,393,131,8,3,0,NaN,257
3,2020/1/23,830,25,34,177,1072,259,8,6,0,NaN,680
4,2020/1/24,1287,41,38,237,1965,457,16,4,0,NaN,1118


In [18]:
# 查看数据的统计信息
alltime_China.describe()

,累计确诊,累计死亡,累计治愈,累计重症,累计疑似,当日新增确诊,当日新增死亡,当日新增治愈,当日新增重症,当日现存确诊,当日新增疑似
count,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.0,0.0,71.000000
mean,58272.267606,2025.056338,33245.281690,4796.183099,6094.830986,1148.140845,46.549296,1071.154930,0.0,NaN,1378.154930
std,30572.980946,1264.722329,29864.644742,3724.337603,8497.722887,1985.171351,43.866612,930.775616,0.0,NaN,1749.404925
min,291.000000,6.000000,25.000000,0.000000,37.000000,11.000000,1.000000,0.000000,0.0,NaN,17.000000
25%,32853.500000,679.000000,1795.000000,1550.000000,178.500000,46.500000,9.500000,302.000000,0.0,NaN,42.500000
50%,77658.000000,2663.000000,27323.000000,4020.000000,1418.000000,397.000000,30.000000,819.000000,0.0,NaN,439.000000
75%,80818.500000,3182.500000,64826.000000,7514.500000,8598.500000,1934.500000,73.000000,1679.500000,0.0,NaN,2363.500000
max,81518.000000,3305.000000,76052.000000,11977.000000,28942.000000,14108.000000,150.000000,3622.000000,0.0,NaN,5328.000000


In [19]:
# 缺失值处理

# 计算当日现存确诊人数
alltime_China['当日现存确诊'] = alltime_China['累计确诊']-alltime_China['累计治愈']-alltime_China['累计死亡']
## 计算病死率
alltime_China['病死率'] = (alltime_China['累计死亡']/alltime_China['累计确诊']).apply(lambda x: format(x, '.2f')) 
# 将病死率数据类型转换为float
alltime_China['病死率'] = alltime_China['病死率'].astype('float')
# 删除当日新增重症一列
alltime_China.drop(['当日新增重症'],axis=1,inplace=True)
alltime_China.to_csv("alltime_China_to4.1.csv", index=False, sep=',')

In [20]:
x_data=list(alltime_China['日期'].values)
# 将日期改成datetime格式
alltime_China['日期'] = pd.to_datetime(alltime_China['日期'])

# 设置日期为索引
alltime_China.set_index('日期',inplace=True)     # 也可使用pd.read_csv("./alltime_China_2020_03_30.csv",parse_dates=['date'],index_col='date')

In [21]:
alltime_China

,累计确诊,累计死亡,累计治愈,累计重症,累计疑似,当日新增确诊,当日新增死亡,当日新增治愈,当日现存确诊,当日新增疑似,病死率
日期,,,,,,,,,,,
2020-01-20,291,6,25,0,54,291,6,25,260,27,0.02
2020-01-21,440,9,25,102,37,149,3,0,406,26,0.02
2020-01-22,571,17,28,95,393,131,8,3,526,257,0.03
2020-01-23,830,25,34,177,1072,259,8,6,771,680,0.03
2020-01-24,1287,41,38,237,1965,457,16,4,1208,1118,0.03
...,...,...,...,...,...,...,...,...,...,...,...
2020-03-26,81340,3292,74588,1034,189,55,5,537,3460,49,0.04
2020-03-27,81394,3295,74971,921,184,54,3,383,3128,29,0.04
2020-03-28,81439,3300,75448,742,174,45,5,477,2691,28,0.04


In [22]:
name=alltime_China.columns.values.tolist()
print(list(alltime_China[name[0]].values))
print(list(alltime_China[name[1]].values))
print(list(alltime_China[name[2]].values))
print(list(alltime_China[name[3]].values))
print(list(alltime_China[name[4]].values))
print(list(alltime_China[name[5]].values))
print(list(alltime_China[name[6]].values))
print(list(alltime_China[name[7]].values))
print(list(alltime_China[name[8]].values))
print(list(alltime_China[name[9]].values))

[291, 440, 571, 830, 1287, 1975, 2744, 4515, 5974, 7711, 9692, 11791, 14380, 17205, 20438, 24324, 28018, 31161, 34546, 37198, 40171, 42638, 44653, 58761, 63851, 66492, 68500, 70548, 72436, 74185, 75002, 75891, 76288, 76741, 77150, 77658, 78064, 78497, 78824, 79251, 79824, 80026, 80151, 80270, 80409, 80552, 80651, 80695, 80735, 80754, 80778, 80793, 80813, 80824, 80844, 80860, 80881, 80894, 80928, 80967, 81008, 81054, 81093, 81171, 81218, 81285, 81340, 81394, 81439, 81470, 81518]
[6, 9, 17, 25, 41, 56, 80, 106, 132, 170, 213, 259, 304, 361, 425, 490, 563, 636, 722, 811, 908, 1016, 1113, 1259, 1380, 1523, 1665, 1770, 1868, 2004, 2118, 2236, 2345, 2442, 2592, 2663, 2715, 2744, 2788, 2835, 2870, 2912, 2943, 2981, 3012, 3042, 3070, 3097, 3119, 3136, 3158, 3169, 3176, 3189, 3199, 3213, 3226, 3237, 3245, 3248, 3255, 3261, 3270, 3277, 3281, 3287, 3292, 3295, 3300, 3304, 3305]
[25, 25, 28, 34, 38, 49, 51, 60, 103, 124, 171, 243, 328, 475, 632, 892, 1153, 1540, 2050, 2649, 3281, 3996, 4740, 5642,

In [37]:
#dataframe 获取列名列表
name=alltime_China.columns.values.tolist()
#x_datax=[d.replace("2020/", "") for d in x_data]
map_4=(
    Line()
    .add_xaxis(xaxis_data=x_data)
    .add_yaxis(
        series_name=name[0],
        stack="总量",
        y_axis=[291, 440, 571, 830, 1287, 1975, 2744, 4515, 5974, 7711, 9692, 11791, 14380, 17205, 20438, 24324, 28018, 31161, 34546, 37198, 40171, 42638, 44653, 58761, 63851, 66492, 68500, 70548, 72436, 74185, 75002, 75891, 76288, 76741, 77150, 77658, 78064, 78497, 78824, 79251, 79824, 80026, 80151, 80270, 80409, 80552, 80651, 80695, 80735, 80754, 80778, 80793, 80813, 80824, 80844, 80860, 80881, 80894, 80928, 80967, 81008, 81054, 81093, 81171, 81218, 81285, 81340, 81394, 81439, 81470, 81518],
        #areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
        #is_smooth=True,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name=name[1],
        stack="总量",
        y_axis=[6, 9, 17, 25, 41, 56, 80, 106, 132, 170, 213, 259, 304, 361, 425, 490, 563, 636, 722, 811, 908, 1016, 1113, 1259, 1380, 1523, 1665, 1770, 1868, 2004, 2118, 2236, 2345, 2442, 2592, 2663, 2715, 2744, 2788, 2835, 2870, 2912, 2943, 2981, 3012, 3042, 3070, 3097, 3119, 3136, 3158, 3169, 3176, 3189, 3199, 3213, 3226, 3237, 3245, 3248, 3255, 3261, 3270, 3277, 3281, 3287, 3292, 3295, 3300, 3304, 3305],
        #areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
        #is_smooth=True,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name=name[2],
        stack="总量",
        y_axis=[25, 25, 28, 34, 38, 49, 51, 60, 103, 124, 171, 243, 328, 475, 632, 892, 1153, 1540, 2050, 2649, 3281, 3996, 4740, 5642, 6723, 8096, 9419, 10844, 12552, 14376, 16157, 18266, 20659, 22888, 24734, 27323, 29745, 32495, 36117, 39002, 41625, 44462, 47204, 49856, 52045, 53726, 55404, 57065, 58600, 59897, 61475, 62793, 64111, 65541, 66911, 67749, 68679, 69601, 70420, 71150, 71740, 72244, 72703, 73159, 73650, 74051, 74588, 74971, 75448, 75770, 76052],
        #areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
        #is_smooth=True,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name=name[3],
        stack="总量",
        y_axis=[0, 102, 95, 177, 237, 324, 461, 976, 1239, 1370, 1527, 1795, 2110, 2296, 2788, 3219, 3859, 4821, 6101, 6188, 6484, 7333, 8204, 8030, 10204, 11053, 11272, 10644, 11741, 11977, 11864, 11633, 11477, 10968, 9915, 9126, 8752, 8346, 7952, 7664, 7365, 7110, 6806, 6416, 5952, 5737, 5489, 5264, 5111, 4794, 4492, 4257, 4020, 3610, 3226, 3032, 2830, 2622, 2314, 2136, 1963, 1845, 1749, 1573, 1399, 1235, 1034, 921, 742, 633, 528],
        #areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
        #is_smooth=True,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name=name[4],
        stack="总量",
        y_axis=[54, 37, 393, 1072, 1965, 2684, 5794, 6973, 9239, 12167, 15238, 17988, 19544, 21558, 23214, 23260, 24702, 26359, 27657, 28942, 23589, 21675, 16067, 13435, 10109, 8969, 8228, 7264, 6242, 5248, 4922, 5206, 5365, 4148, 3434, 2824, 2491, 2358, 2308, 1418, 851, 715, 587, 520, 522, 482, 502, 458, 421, 349, 285, 253, 147, 115, 113, 134, 128, 119, 105, 104, 106, 118, 136, 132, 134, 159, 189, 184, 174, 168, 183],
        #areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name=name[5],
        stack="总量",
        y_axis=[291, 149, 131, 259, 457, 688, 769, 1771, 1459, 1737, 1981, 2099, 2589, 2825, 3233, 3886, 3694, 3143, 3385, 2652, 2973, 2467, 2015, 14108, 5090, 2641, 2008, 2048, 1888, 1749, 817, 889, 397, 453, 409, 508, 406, 433, 327, 427, 573, 202, 125, 119, 139, 143, 99, 44, 40, 19, 24, 15, 20, 11, 20, 16, 21, 13, 34, 39, 41, 46, 39, 78, 47, 67, 55, 54, 45, 31, 48],
        #areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
        #is_smooth=True,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name=name[6],
        stack="总量",
        y_axis=[6, 3, 8, 8, 16, 15, 24, 26, 26, 38, 43, 46, 45, 57, 64, 65, 73, 73, 86, 89, 97, 108, 97, 146, 121, 143, 142, 105, 98, 136, 114, 118, 109, 97, 150, 71, 52, 29, 44, 47, 35, 42, 31, 38, 31, 30, 28, 27, 22, 17, 22, 11, 7, 13, 10, 14, 13, 11, 8, 3, 7, 6, 9, 7, 4, 6, 5, 3, 5, 4, 1],
        #areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
        #is_smooth=True,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name=name[7],
        stack="总量",
        y_axis=[25, 0, 3, 6, 4, 11, 2, 9, 43, 21, 47, 72, 85, 147, 157, 260, 261, 387, 510, 599, 632, 715, 744, 902, 1081, 1373, 1323, 1425, 1708, 1824, 1781, 2109, 2393, 2229, 1846, 2589, 2422, 2750, 3622, 2885, 2623, 2837, 2742, 2652, 2189, 1681, 1678, 1661, 1535, 1297, 1578, 1318, 1318, 1430, 1370, 838, 930, 922, 819, 730, 590, 504, 459, 456, 491, 401, 537, 383, 477, 322, 282],
        #areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
        is_smooth=True,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name=name[8],
        stack="总量",
        y_axis=[260, 406, 526, 771, 1208, 1870, 2613, 4349, 5739, 7417, 9308, 11289, 13748, 16369, 19381, 22942, 26302, 28985, 31774, 33738, 35982, 37626, 38800, 51860, 55748, 56873, 57416, 57934, 58016, 57805, 56727, 55389, 53284, 51411, 49824, 47672, 45604, 43258, 39919, 37414, 35329, 32652, 30004, 27433, 25352, 23784, 22177, 20533, 19016, 17721, 16145, 14831, 13526, 12094, 10734, 9898, 8976, 8056, 7263, 6569, 6013, 5549, 5120, 4735, 4287, 3947, 3460, 3128, 2691, 2396, 2161],
        #areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
        #is_smooth=True,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name=name[9],
        stack="总量",
        y_axis=[27, 26, 257, 680, 1118, 1309, 3806, 2077, 3248, 4148, 4812, 5019, 4562, 5173, 5072, 3971, 5328, 4833, 4214, 3916, 4008, 3536, 3342, 2807, 2450, 2277, 1918, 1563, 1432, 1185, 1277, 1614, 1361, 882, 620, 530, 439, 508, 452, 248, 132, 141, 129, 143, 143, 102, 99, 84, 60, 36, 31, 33, 33, 17, 39, 41, 45, 21, 23, 31, 36, 45, 47, 35, 33, 58, 49, 29, 28, 17, 44],
        #areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
        #is_smooth=True,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="全国新冠肺炎数据折线图", pos_left="center"),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        legend_opts=opts.LegendOpts( 
            orient="vertical", pos_top="20%", pos_left="88%" #图例设置
        ),
        #yaxis_opts=opts.AxisOpts(
         #   type_="value",
         #   axistick_opts=opts.AxisTickOpts(is_show=True),
         #   splitline_opts=opts.SplitLineOpts(is_show=True),
        #),
        #xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),

        
        #给x轴添加伸缩条
        datazoom_opts=[opts.DataZoomOpts()],
    )
    )
map_4.render("全国新冠肺炎数据折线图.html")
map_4.render_notebook()

In [24]:
# 时间序列数据绘制折线图
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import matplotlib.ticker as ticker
import datetime

fig, ax = plt.subplots(figsize=(8,4))

alltime_China.plot(marker='o',ms=2,lw=1,ax=ax)

ax.xaxis.set_major_locator(dates.MonthLocator())    #设置间距
ax.xaxis.set_major_formatter(dates.DateFormatter('%b'))    #设置日期格式

fig.autofmt_xdate()    #自动调整日期倾斜

# 图例位置调整
plt.legend(bbox_to_anchor = [1,1])

plt.title('全国新冠肺炎数据折线图',size=15)
plt.ylabel('人数')
plt.grid(axis='y')
plt.box(False)
plt.show()

<Figure size 800x400 with 1 Axes>

In [25]:
#3世界各国历史数据探索性分析
# 创建中文列名字典
name_dict = {'date':'日期','name':'名称','id':'编号','lastUpdateTime':'更新时间',
             'today_confirm':'当日新增确诊','today_suspect':'当日新增疑似',
             'today_heal':'当日新增治愈','today_dead':'当日新增死亡',
             'today_severe':'当日新增重症','today_storeConfirm':'当日现存确诊',
             'total_confirm':'累计确诊','total_suspect':'累计疑似',
             'total_heal':'累计治愈','total_dead':'累计死亡','total_severe':'累计重症'}

# 更改列名
alltime_world.rename(columns=name_dict,inplace=True)
alltime_world.head()

,日期,累计确诊,累计疑似,累计治愈,累计死亡,累计重症,当日新增确诊,当日新增疑似,当日新增治愈,当日新增死亡,当日新增重症,当日现存确诊,名称
0,2020-03-03,1,0,0,0,0,1,0.0,0,0,NaN,NaN,突尼斯
1,2020-03-08,2,0,0,0,0,1,0.0,0,0,0.0,NaN,突尼斯
2,2020-03-09,5,0,0,0,0,3,0.0,0,0,0.0,NaN,突尼斯
3,2020-03-11,7,0,0,0,0,2,0.0,0,0,0.0,NaN,突尼斯
4,2020-03-12,13,0,0,0,0,6,0.0,0,0,0.0,NaN,突尼斯


In [26]:
#dataframe 获取时间列名列表
#4月1日数据统计不完全，我们用将其删除
x_wdata=[d.replace("2020-", "") for d in list(alltime_world['日期'].value_counts().sort_index().index)][0:-1]
# 将日期一列数据类型变为datetime
alltime_world['日期'] = pd.to_datetime(alltime_world['日期'])

# 计算当日现存确诊
alltime_world['当日现存确诊'] = alltime_world['累计确诊']-alltime_world['累计治愈']-alltime_world['累计死亡']
alltime_world.to_csv("alltime_world_to4.1.csv", index=False, sep=',')

In [27]:
# 统计每天有多少国家出现疫情
alltime_world['日期'].value_counts().sort_index()
print(list(alltime_world['日期'].value_counts().sort_index().values)[0:-1])

[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 3, 3, 2, 2, 2, 4, 4, 4, 25, 6, 9, 5, 6, 9, 6, 8, 4, 6, 6, 6, 6, 10, 8, 10, 13, 21, 22, 31, 45, 33, 32, 41, 40, 43, 49, 52, 63, 64, 64, 73, 71, 70, 105, 113, 102, 94, 86, 94, 109, 106, 96, 129, 116, 118, 125, 157, 130, 120, 110, 147, 112]


In [28]:
import pyecharts.options as opts
from pyecharts.charts import Line
map_5=(Line()
    .add_xaxis(xaxis_data=x_wdata)
    .add_yaxis(
        series_name="疫情国家数量",
        y_axis=[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 3, 3, 2, 2, 2, 4, 4, 4, 25, 6, 9, 5, 6, 9, 6, 8, 4, 6, 6, 6, 6, 10, 8, 10, 13, 21, 22, 31, 45, 33, 32, 41, 40, 43, 49, 52, 63, 64, 64, 73, 71, 70, 105, 113, 102, 94, 86, 94, 109, 106, 96, 129, 116, 118, 125, 157, 130, 120, 110, 147, 112],
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="最大值"),
                opts.MarkPointItem(type_="min", name="最小值"),
            ]
        ),
        markline_opts=opts.MarkLineOpts(
            data=[opts.MarkLineItem(type_="average", name="平均值")]
        ),
    )
  .set_global_opts(
        title_opts=opts.TitleOpts(title="疫情国家数量变化", subtitle="截止日期2020-3-31"),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        toolbox_opts=opts.ToolboxOpts(is_show=True),
        xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
    )
    )
map_5.render("疫情国家数量变化.html")
map_5.render_notebook()

In [29]:
# 设置日期索引
alltime_world.set_index('日期',inplace=True)
#选取多国数据分析
# groupby创建层次化索引
data = alltime_world.groupby(['日期','名称']).mean()
# 提取部分数据(中国、韩国、美国、意大利、英国、西班牙、德国)
data_part = data.loc(axis=0)[:,['中国','日本','韩国','美国','意大利','英国','西班牙','德国']]
# 再将层级索引（国家名称）还原
data_part.reset_index(['名称','日期'],inplace=True)
data_part.to_csv("部分热门国家数据.csv", index=False, sep=',')
data_part.head()

,日期,名称,累计确诊,累计疑似,累计治愈,累计死亡,累计重症,当日新增确诊,当日新增疑似,当日新增治愈,当日新增死亡,当日新增重症,当日现存确诊
0,2020-01-20,韩国,1,0,0,0,0,1,0.0,0,0,NaN,1
1,2020-01-21,韩国,1,0,0,0,0,0,0.0,0,0,NaN,1
2,2020-01-22,韩国,1,0,0,0,0,0,0.0,0,0,NaN,1
3,2020-01-23,韩国,1,0,0,0,0,0,0.0,0,0,NaN,1
4,2020-01-24,韩国,2,0,0,0,0,1,0.0,0,0,NaN,2


In [30]:
print('1  ',list(data_part[data_part['名称']=='韩国']['累计确诊'].values))
print('2  ',list(data_part[data_part['名称']=='中国']['累计确诊'].values))
print('3  ',list(data_part[data_part['名称']=='日本']['累计确诊'].values))
print('4  ',list(data_part[data_part['名称']=='美国']['累计确诊'].values))
print('5  ',list(data_part[data_part['名称']=='意大利']['累计确诊'].values))
print('6  ',list(data_part[data_part['名称']=='英国']['累计确诊'].values))
print('7  ',list(data_part[data_part['名称']=='西班牙']['累计确诊'].values))
print('8  ',list(data_part[data_part['名称']=='德国']['累计确诊'].values))

1   [1, 1, 1, 1, 2, 2, 3, 4, 4, 4, 6, 11, 12, 15, 15, 16, 19, 23, 24, 24, 27, 27, 28, 28, 28, 28, 28, 29, 30, 31, 51, 104, 204, 433, 602, 833, 977, 1261, 1766, 2337, 3148, 4212, 4812, 5328, 5766, 6284, 6767, 7134, 7382, 7513, 7755, 7869, 7979, 8086, 8162, 8236, 8320, 8413, 8565, 8652, 8799, 8897, 8961, 9037, 9137, 9241, 9332, 9478, 9583, 9661, 9786, 9887]
2   [24433, 31161, 34598, 37162, 40224, 42708, 44730, 58839, 63932, 66575, 68584, 70635, 72528, 74279, 75101, 75993, 76392, 76846, 77262, 77779, 78190, 78630, 78959, 79389, 79968, 80174, 80302, 80422, 80565, 80710, 80813, 80859, 80904, 80924, 80955, 80992, 81003, 81021, 81048, 81078, 81116, 81151, 81235, 81300, 81416, 81498, 81600, 81747, 81846, 81960, 82078, 82213, 82341, 82448, 82545, 82631, 82640]
3   [4, 6, 15, 20, 30, 45, 86, 146, 146, 146, 148, 203, 251, 259, 340, 413, 500, 616, 705, 728, 739, 763, 838, 850, 862, 891, 919, 933, 947, 961, 980, 999, 1036, 1056, 1112, 1157, 1190, 1218, 1277, 1334, 1387, 1408, 1484, 1530, 1547, 1594

In [31]:
print('1  ',list(data_part[data_part['名称']=='韩国']['当日新增确诊'].values))
print('2  ',list(data_part[data_part['名称']=='中国']['当日新增确诊'].values))
print('3  ',list(data_part[data_part['名称']=='日本']['当日新增确诊'].values))
print('4  ',list(data_part[data_part['名称']=='美国']['当日新增确诊'].values))
print('5  ',list(data_part[data_part['名称']=='意大利']['当日新增确诊'].values))
print('6  ',list(data_part[data_part['名称']=='英国']['当日新增确诊'].values))
print('7  ',list(data_part[data_part['名称']=='西班牙']['当日新增确诊'].values))
print('8  ',list(data_part[data_part['名称']=='德国']['当日新增确诊'].values))

1   [1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 2, 5, 1, 3, 0, 1, 3, 4, 1, 0, 3, 0, 1, 0, 0, 0, 0, 1, 1, 1, 20, 53, 100, 229, 169, 231, 144, 284, 505, 571, 811, 1064, 600, 516, 438, 518, 483, 367, 248, 131, 242, 114, 110, 107, 76, 74, 84, 93, 152, 87, 147, 98, 64, 76, 100, 104, 91, 146, 105, 78, 125, 101]
2   [24433, 6728, 3437, 2564, 3062, 2484, 2022, 14109, 5093, 2643, 2009, 2051, 1893, 1751, 822, 892, 399, 454, 416, 517, 411, 440, 329, 430, 579, 206, 128, 120, 143, 145, 103, 46, 45, 20, 31, 37, 11, 18, 27, 30, 38, 35, 84, 65, 116, 82, 102, 147, 99, 114, 118, 135, 128, 107, 97, 86, 9]
3   [4, 2, 9, 5, 10, 15, 41, 60, 0, 0, 2, 55, 48, 8, 81, 73, 87, 116, 89, 23, 11, 24, 75, 12, 12, 29, 28, 14, 14, 14, 19, 19, 37, 20, 56, 45, 33, 28, 59, 57, 53, 21, 76, 46, 17, 47, 37, 43, 53, 40, 48, 40, 10, 161, 91, 102, 292, 97, 93, 244]
4   [14, 1, 19, 19, 0, 4, 3, 0, 3, 0, 26, 7, 26, 41, 69, 92, 121, 102, 170, 287, 308, 346, 546, 612, 958, 668, 1978, 2668, 4262, 6274, 2419, 13181, 8739, 9697, 15300, 16880, 141

In [32]:
from pyecharts import options as opts
from pyecharts.charts import Bar, Grid, Line

line1 = (
    Line()
    .add_xaxis(xaxis_data=x_wdata)
    .add_yaxis(
        series_name="韩国", 
        y_axis=[1, 1, 1, 1, 2, 2, 3, 4, 4, 4, 6, 11, 12, 15, 15, 16, 19, 23, 24, 24, 27, 27, 28, 28, 28, 28, 28, 29, 30, 31, 51, 104, 204, 433, 602, 833, 977, 1261, 1766, 2337, 3148, 4212, 4812, 5328, 5766, 6284, 6767, 7134, 7382, 7513, 7755, 7869, 7979, 8086, 8162, 8236, 8320, 8413, 8565, 8652, 8799, 8897, 8961, 9037, 9137, 9241, 9332, 9478, 9583, 9661, 9786, 9887],
        #取消数据标签
        label_opts=opts.LabelOpts(is_show=False),)
    .add_yaxis(
        series_name="中国", 
        y_axis=[24433, 31161, 34598, 37162, 40224, 42708, 44730, 58839, 63932, 66575, 68584, 70635, 72528, 74279, 75101, 75993, 76392, 76846, 77262, 77779, 78190, 78630, 78959, 79389, 79968, 80174, 80302, 80422, 80565, 80710, 80813, 80859, 80904, 80924, 80955, 80992, 81003, 81021, 81048, 81078, 81116, 81151, 81235, 81300, 81416, 81498, 81600, 81747, 81846, 81960, 82078, 82213, 82341, 82448, 82545, 82631, 82640],
        label_opts=opts.LabelOpts(is_show=False),)
    .add_yaxis(
        series_name="日本", 
        y_axis=[4, 6, 15, 20, 30, 45, 86, 146, 146, 146, 148, 203, 251, 259, 340, 413, 500, 616, 705, 728, 739, 763, 838, 850, 862, 891, 919, 933, 947, 961, 980, 999, 1036, 1056, 1112, 1157, 1190, 1218, 1277, 1334, 1387, 1408, 1484, 1530, 1547, 1594, 1631, 1674, 1727, 1767, 1815, 1855, 1865, 2026, 2117, 2219, 2511, 2608, 2701, 2945],
        label_opts=opts.LabelOpts(is_show=False),)
    .add_yaxis(
        series_name="美国", 
        y_axis=[14, 15, 34, 53, 53, 57, 60, 60, 63, 63, 89, 96, 122, 163, 232, 324, 445, 547, 717, 1004, 1312, 1658, 2204, 2816, 3774, 4442, 6420, 9088, 13350, 19624, 22043, 35224, 43963, 53660, 68960, 85840, 100014, 114763, 140917, 164254, 189510],
        label_opts=opts.LabelOpts(is_show=False),)
    .add_yaxis(
        series_name="意大利", 
        y_axis=[2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 21, 79, 132, 219, 283, 400, 650, 888, 1128, 1694, 2084, 2502, 3144, 3927, 4636, 6030, 7522, 9220, 10149, 12598, 15385, 17760, 21270, 24938, 27980, 31506, 35713, 41035, 47021, 53578, 59138, 63927, 69176, 74386, 80539, 86498, 92472, 97689, 101739, 105792],
        label_opts=opts.LabelOpts(is_show=False),)
    .add_yaxis(
        series_name="英国", 
        y_axis=[8, 9, 13, 16, 20, 23, 36, 40, 51, 85, 116, 163, 206, 273, 319, 382, 460, 590, 798, 1145, 1372, 1543, 1950, 2626, 3269, 3983, 5018, 5683, 6650, 8077, 9529, 11658, 14579, 17089, 19522, 22141, 25481],
        label_opts=opts.LabelOpts(is_show=False),)
    .add_yaxis(
        series_name="西班牙", 
        y_axis=[2, 3, 9, 13, 25, 41, 58, 74, 124, 165, 193, 208, 387, 516, 613, 1223, 1695, 2882, 3146, 4334, 6393, 7753, 9191, 11178, 13716, 17147, 20410, 24926, 28572, 39673, 42058, 47610, 56197, 65719, 73235, 80110, 85195, 94417, 95923],
        label_opts=opts.LabelOpts(is_show=False),)
    .add_yaxis(
        series_name="德国", 
        y_axis=[14, 16, 16, 18, 26, 48, 62, 66, 129, 157, 196, 344, 565, 639, 795, 902, 1139, 1524, 2046, 2745, 3675, 4585, 5813, 6230, 9257, 11973, 13979, 20099, 22364, 27181, 29056, 30138, 37323, 43938, 50871, 53340, 62535, 66885, 71808],
        label_opts=opts.LabelOpts(is_show=False),)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="各国累计确诊病例"),
                toolbox_opts=opts.ToolboxOpts(is_show=True),
        #给x轴添加伸缩条
        datazoom_opts=[
                opts.DataZoomOpts(
                    is_show=False,
                    type_="inside",
                    xaxis_index=[0, 1],
                    range_start=98,
                    range_end=100,
                ),
                opts.DataZoomOpts(
                    is_show=True,
                    xaxis_index=[0, 1],
                    type_="slider",
                    pos_top="95%",
                    range_start=98,
                    range_end=100,
                ),
            ],
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),

                    )
)
line2 = (
    Line()
    .add_xaxis(xaxis_data=x_wdata)
    .add_yaxis(
        series_name="韩国", 
        y_axis=[1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 2, 5, 1, 3, 0, 1, 3, 4, 1, 0, 3, 0, 1, 0, 0, 0, 0, 1, 1, 1, 20, 53, 100, 229, 169, 231, 144, 284, 505, 571, 811, 1064, 600, 516, 438, 518, 483, 367, 248, 131, 242, 114, 110, 107, 76, 74, 84, 93, 152, 87, 147, 98, 64, 76, 100, 104, 91, 146, 105, 78, 125, 101],
        label_opts=opts.LabelOpts(is_show=False),)
    
    .add_yaxis(
        series_name="中国", 
        y_axis=[24433, 6728, 3437, 2564, 3062, 2484, 2022, 14109, 5093, 2643, 2009, 2051, 1893, 1751, 822, 892, 399, 454, 416, 517, 411, 440, 329, 430, 579, 206, 128, 120, 143, 145, 103, 46, 45, 20, 31, 37, 11, 18, 27, 30, 38, 35, 84, 65, 116, 82, 102, 147, 99, 114, 118, 135, 128, 107, 97, 86, 9],
        label_opts=opts.LabelOpts(is_show=False),)
    
    .add_yaxis(
        series_name="日本", 
        y_axis=[4, 2, 9, 5, 10, 15, 41, 60, 0, 0, 2, 55, 48, 8, 81, 73, 87, 116, 89, 23, 11, 24, 75, 12, 12, 29, 28, 14, 14, 14, 19, 19, 37, 20, 56, 45, 33, 28, 59, 57, 53, 21, 76, 46, 17, 47, 37, 43, 53, 40, 48, 40, 10, 161, 91, 102, 292, 97, 93, 244],
        label_opts=opts.LabelOpts(is_show=False),)
    
    .add_yaxis(
        series_name="美国", 
        y_axis=[14, 1, 19, 19, 0, 4, 3, 0, 3, 0, 26, 7, 26, 41, 69, 92, 121, 102, 170, 287, 308, 346, 546, 612, 958, 668, 1978, 2668, 4262, 6274, 2419, 13181, 8739, 9697, 15300, 16880, 14174, 14749, 26154, 23337, 25256],
        label_opts=opts.LabelOpts(is_show=False),)
    .add_yaxis(
        series_name="意大利", 
        y_axis=[2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 17, 58, 53, 87, 64, 117, 250, 238, 240, 566, 390, 418, 642, 783, 709, 1394, 1492, 1698, 929, 2449, 2787, 2375, 3510, 3668, 3042, 3526, 4207, 5322, 5986, 6557, 5560, 4789, 5249, 5210, 6153, 5959, 5974, 5217, 4050, 4053],
        label_opts=opts.LabelOpts(is_show=False),)
    .add_yaxis(
        series_name="英国", 
        y_axis=[8, 1, 4, 3, 4, 3, 13, 4, 11, 34, 31, 47, 43, 67, 46, 63, 78, 130, 208, 347, 227, 171, 407, 676, 643, 714, 1035, 665, 967, 1427, 1452, 2129, 2921, 2510, 2433, 2619, 3340],
        label_opts=opts.LabelOpts(is_show=False),)
    .add_yaxis(
        series_name="西班牙", 
        y_axis=[2, 1, 6, 4, 12, 16, 17, 16, 50, 41, 28, 15, 179, 129, 97, 610, 472, 1187, 264, 1188, 2059, 1360, 1438, 1987, 2538, 3431, 3263, 4516, 3646, 11101, 2385, 5552, 8587, 9522, 7516, 6875, 5085, 9222, 1506],
        label_opts=opts.LabelOpts(is_show=False),)
    .add_yaxis(
        series_name="德国", 
        y_axis=[14, 2, 0, 2, 8, 22, 14, 4, 63, 28, 39, 148, 221, 74, 156, 107, 237, 385, 522, 699, 930, 910, 1228, 417, 3027, 2716, 2006, 6120, 2265, 4817, 1875, 1082, 7185, 6615, 6933, 2469, 9195, 4350, 4923],
        label_opts=opts.LabelOpts(is_show=False),)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="各国当日新增确诊病例", pos_top="48%"),
        legend_opts=opts.LegendOpts(pos_top="48%"),
        datazoom_opts=[opts.DataZoomOpts(type_="inside")],

    )
)

grid = (
    Grid()
    .add(line1, grid_opts=opts.GridOpts(pos_bottom="60%"))
    .add(line2, grid_opts=opts.GridOpts(pos_top="60%"))
)
grid.render("各国新冠肺炎累计确诊&新增确诊病例折线图.html")
grid.render_notebook()

In [33]:
#4.全国各省历史数据
# 创建中文列名字典
name_dict = {'date':'日期','name':'名称','id':'编号','lastUpdateTime':'更新时间',
             'today_confirm':'当日新增确诊','today_suspect':'当日新增疑似',
             'today_heal':'当日新增治愈','today_dead':'当日新增死亡',
             'today_severe':'当日新增重症','today_storeConfirm':'当日现存确诊',
             'total_confirm':'累计确诊','total_suspect':'累计疑似',
             'total_heal':'累计治愈','total_dead':'累计死亡','total_severe':'累计重症'}

# 更改列名
alltime_province.rename(columns=name_dict,inplace=True)
# 计算当日现存确诊
alltime_province['当日现存确诊'] = alltime_province['累计确诊']-alltime_province['累计治愈']-alltime_province['累计死亡']

In [34]:
#筛选出湖北的数据
hubei_data = alltime_province[alltime_province['名称']=='湖北'][['日期','累计死亡','累计确诊']]

#筛选出湖北省外的数据
out_hubei_data = alltime_province[['日期','累计死亡','累计确诊','名称']][alltime_province.名称 != '湖北']
out_hubei_data.reset_index(drop=True, inplace=True)
#根据日期求和
out_hubei_data =out_hubei_data.groupby('日期').sum()
#去掉4月1的数据
out_hubei_data[:-1]
out_hubei_data.reset_index(['日期'],inplace=True)

#探索病死率
out_hubei_data['病死率'] = (out_hubei_data['累计死亡']/out_hubei_data['累计确诊']).apply(lambda x: format(x, '.2f')) 
hubei_data['病死率'] = (hubei_data['累计死亡']/hubei_data['累计确诊']).apply(lambda x: format(x, '.2f')) 
# 将病死率数据类型转换为float
out_hubei_data['病死率'] = out_hubei_data['病死率'].astype('float')
hubei_data['病死率'] = hubei_data['病死率'].astype('float')

In [35]:
hubei_data

,日期,累计死亡,累计确诊,病死率
0,2020-01-20,6,270,0.02
1,2020-01-21,9,375,0.02
2,2020-01-22,17,444,0.04
3,2020-01-23,24,549,0.04
4,2020-01-24,39,729,0.05
...,...,...,...,...
67,2020-03-27,3177,67801,0.05
68,2020-03-28,3182,67801,0.05
69,2020-03-29,3186,67801,0.05
70,2020-03-30,3187,67801,0.05


In [42]:
#list(hubei_data['日期'].values)
in1 =['2020-01-20', '2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29', '2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05', '2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09', '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13', '2020-03-14', '2020-03-15', '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-21', '2020-03-22', '2020-03-23', '2020-03-24', '2020-03-25', '2020-03-26', '2020-03-27', '2020-03-28', '2020-03-29', '2020-03-30', '2020-03-31']
#list(hubei_data['病死率'].values)
in2 = [0.02, 0.02, 0.04, 0.04, 0.05, 0.05, 0.05, 0.04, 0.04, 0.04, 0.04, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]
#list(out_hubei_data['日期'].values)
out1 = ['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29', '2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05', '2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09', '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13', '2020-03-14', '2020-03-15', '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-21', '2020-03-22', '2020-03-23', '2020-03-24', '2020-03-25', '2020-03-26', '2020-03-27', '2020-03-28', '2020-03-29', '2020-03-30', '2020-03-31', '2020-04-01']
#list(out_hubei_data['病死率'].values)
out2 = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
#list(alltime_China['病死率'].values)
all_cn = [0.02, 0.02, 0.03, 0.03, 0.03, 0.03, 0.03, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04]
line_in=(
    Line()
    .add_xaxis(xaxis_data=in1)
    .add_yaxis(
        series_name="湖北省内",
        y_axis=in2,
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="最大值"),
                opts.MarkPointItem(type_="min", name="最小值"),
            ]
        ),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name="全国",
        y_axis=all_cn,
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="最大值"),
                opts.MarkPointItem(type_="min", name="最小值"),
            ]
        ),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="病死率折线图"),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        toolbox_opts=opts.ToolboxOpts(is_show=True),
        xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
        datazoom_opts=[opts.DataZoomOpts()],
    )
        )

line_out=(
    Line()
    .add_xaxis(xaxis_data=out1)
    .add_yaxis(
        series_name="湖北省外",
        y_axis=out2,
        markline_opts=opts.MarkLineOpts(
            data=[
                opts.MarkLineItem(symbol="none", x="90%", y="max"),
            ]
        ),
        label_opts=opts.LabelOpts(is_show=False),
    )
        )

line_in.overlap(line_out)
line_in.render('病死率折线图.html')
line_in.render_notebook()